## Config connection to Apache Spark local

In [1]:
import findspark
findspark.init(spark_home = "/home/thanhphat/BigData/spark-3.5.0-bin-hadoop3")

In [2]:
from pyspark.sql import SparkSession

import traceback
import pyspark.sql.functions as f
import pyspark.sql.types as t

In [3]:
project_name = "Global_Electronics_Retailer"

In [4]:
# Config
    # Threads: 4 CPU
    # Dirver: 1
    # Number of executor: 3
    # 1 CPU for each executor
    # 1g memory for each executor


# Create SparkSession
spark = SparkSession.builder.master("local[4]") \
                            .appName("Source_to_Bronze") \
                            .config("spark.sql.parquet.vorder.enabled", "true") \
                            .config("spark.sql.shuffle.partitions", 100) \
                            .config("spark.driver.memory", "1g") \
                            .config("spark.executor.instances", "3") \
                            .config("spark.executor.cores", "1") \
                            .config("spark.executor.memory", "1g") \
                            .config("spark.jars", "../driver/mysql-connector-j-8.1.0.jar") \
                            .getOrCreate()

spark.sparkContext.getConf().getAll()

24/06/28 22:42:50 WARN Utils: Your hostname, thanhphat-inspiron-5406-2n1 resolves to a loopback address: 127.0.1.1; using 192.168.1.8 instead (on interface wlp0s20f3)
24/06/28 22:42:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/06/28 22:42:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


[('spark.driver.port', '41959'),
 ('spark.executor.memory', '2g'),
 ('spark.master', 'local[4]'),
 ('spark.driver.host', '192.168.1.8'),
 ('spark.app.startTime', '1719589371736'),
 ('spark.app.submitTime', '1719589371545'),
 ('spark.executor.id', 'driver'),
 ('spark.executor.cores', '2'),
 ('spark.jars', '../driver/mysql-connector-j-8.1.0.jar'),
 ('spark.sql.shuffle.partitions', '100'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base

In [5]:
import sys
sys.path.append("/home/thanhphat/PersonalProject/Global_Electronics_Retailer/source")

from modules.Extraction import *
from modules.HDFSUtils import *
from modules.LogUtils import *
from modules.Metadata import *


# Instance for modules
extraction = Extraction()
hdfsUtils = HDFSUtils()
logUtils = LogUtils() 
metadata = Metadata()

# Define base_path
file_path = f"hdfs://localhost:9000/lakehouse/LH_{project_name}/Files/Bronze"
log_path = f"hdfs://localhost:9000/lakehouse/LH_{project_name}/Files/log"

In [6]:
executionDate = str(spark.sql("SELECT CURRENT_DATE()").collect()[0][0])

# Partition Execution Date
parse_execution = executionDate.split("-")
year = parse_execution[0]
month = parse_execution[1]
day = parse_execution[2]

In [7]:
# # Define parameter for connect to MySQL
# database = "Global_Electronics_Retailer"
# dbname = f"jdbc:mysql://localhost:3306/{database}"
# driver = "com.mysql.jdbc.Driver"
# username = "root"
# password = "password"

# df = extraction.read_table_mysql(spark, driver, dbname, "customers", username, password)

# display(df)

## Metadata Table Action

In [8]:
# Read metadata action
# metadata_action = metadata.read_metadata_action("admin", "admin", "metadata", "config_table", \
#                                                 "CusDB -> Bronze")

from airflow.models import Variable
metadata_action = Variable.get(key = "metadata_action", deserialize_json=True, default_var=None)

# Define for log job
batch_run = hdfsUtils.check_batch_run(project_name, executionDate)
start_time = ""
end_time = ""
error = ""
status = ""
source_row_read = 0
numInserted = 0
numUpdated = 0


# Define parameter for connect to MySQL
database = "Global_Electronics_Retailer"
dbname = f"jdbc:mysql://localhost:3306/{database}"
driver = "com.mysql.jdbc.Driver"
username = "root"
password = "password"


# tblNames
tblNames = ["customers", "sales", "products", "stores", "exchange_rates"]


# # Read all table
for metadata in metadata_action:

    task_id = metadata["task_id"]
    task_name = metadata["task_name"]
    source_connection = metadata["source_connection"]
    source_database = metadata["source_database"]
    source_table = metadata["source_table"].lower()
    phase = metadata["phase"]

    # Start time for check
    start_time = spark.sql(''' SELECT CURRENT_TIMESTAMP() as current_time ''') \
                        .collect()[0]["current_time"].strftime('%Y-%m-%d %H:%M:%S')
    try:
        # Read data
        df = extraction.read_table_mysql(spark, driver, dbname, source_table, username, password)
        

        # Validate data
        source_row_read = df.count()
        numInserted = df.count()

        # Create new column for partition
        df = extraction.create_year_month_day(df, executionDate, f)
        
        # Display df
        # df.show()

        # Write data to HDFS
        code = hdfsUtils.check_exist_data(executionDate, project_name, source_table)
        # Exist file
        if code == 0: # Yes => Append for version data
            df.write.mode("append").format("parquet") \
                    .save(f"{file_path}/{source_table}/year={year}/month={month}/day={day}/{source_table}_{year}_{month}_{day}-version_{batch_run}.parquet")
        else: # No => First run
            df.write.mode("overwrite").format("parquet") \
                    .save(f"{file_path}/{source_table}/year={year}/month={month}/day={day}/{source_table}_{year}_{month}_{day}-version_{batch_run}.parquet")
    
    except:
        error = traceback.format_exc()
        status = "Failed"

        print("Task ", task_id, " ", status)

    else:
        error = ""
        status = "Success"
        print("Task ", task_id, " ", status)
    
    # End time for check
    end_time = spark.sql(''' SELECT CURRENT_TIMESTAMP() as current_time ''') \
                        .collect()[0]["current_time"].strftime('%Y-%m-%d %H:%M:%S')

    # Check status
    # print("Tablename: ", tblName, "Error: ", error, "Status: ", status, 
    #       "Source rows: ", source_row_read, "Num of rows Inserted: ", numInserted)


    df_log = logUtils.log_data(batch_run, task_name, source_connection, source_database, source_table, "parquet",
                 start_time, end_time, source_row_read, numInserted, numUpdated, "", 
                 "", error, status, phase, t, spark)

    df_log.write.mode("append").format("parquet").save(f"{log_path}/{executionDate}/batch_{batch_run}/")

Connected successfully!!!


ls: Call From thanhphat-Inspiron-5406-2n1/127.0.1.1 to localhost:9000 failed on connection exception: java.net.ConnectException: Connection refused; For more details see:  http://wiki.apache.org/hadoop/ConnectionRefused
Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


Task  1   Failed


Py4JJavaError: An error occurred while calling o163.save.
: java.net.ConnectException: Call From thanhphat-inspiron-5406-2n1/127.0.1.1 to localhost:9000 failed on connection exception: java.net.ConnectException: Connection refused; For more details see:  http://wiki.apache.org/hadoop/ConnectionRefused
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at org.apache.hadoop.net.NetUtils.wrapWithMessage(NetUtils.java:913)
	at org.apache.hadoop.net.NetUtils.wrapException(NetUtils.java:828)
	at org.apache.hadoop.ipc.Client.getRpcResponse(Client.java:1616)
	at org.apache.hadoop.ipc.Client.call(Client.java:1558)
	at org.apache.hadoop.ipc.Client.call(Client.java:1455)
	at org.apache.hadoop.ipc.ProtobufRpc